<a href="https://colab.research.google.com/github/mridul-eecs/signal-processing-apachesparkml-apachesystemml/blob/master/HumanMotionPrimitivesDetection_ETA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Notebook: Extract Transform Load


In [0]:

# spark dependencies:
# citation: http://medium.com/@rmache/big-data-with-spark-in-google-colab-7c046e24b3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 60kB/s 
     |████████████████████████████████| 204kB 49.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=3873d546831d484ed557a52ebf9f1aad1ae6ebb8bf4d804616aa5202d140e356
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
!git clone https://github.com/wchill/HMP_Dataset.git

Cloning into 'HMP_Dataset'...
remote: Enumerating objects: 865, done.
remote: Total 865 (delta 0), reused 0 (delta 0), pack-reused 865
Receiving objects: 100% (865/865), 1010.96 KiB | 12.04 MiB/s, done.


In [0]:
!pip install pixiedust

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 798kB 48.5MB/s 
  Created wheel for pixiedust: filename=pixiedust-1.1.18-cp36-none-any.whl size=321728 sha256=ce95df1c0a173aad5b444a27e2b3a893b6dfc0122842a72c523f0ca7b877d022
  Stored in directory: /root/.cache/pip/wheels/e8/b1/86/c2f2e16e6bf9bfe556f9dbf8adb9f41816c476d73078c7d0eb
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=05b1051f129e522fe63249859e37f9aa78deadc5e8c29b43c661dac7bceeba00
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
Successfully built pixiedust mpld3


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType
from pyspark.sql.functions import lit
import os
from tqdm import tqdm_notebook as tqdm
import pixiedust
pixiedust.get_ipython()

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


import findspark
findspark.init()

APP_NAME= "Human Motion Premitives"
SPARK_URL= "local[*]"
RANDOM_SEED = 141109
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 8
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

In [0]:
import os
basedir= 'HMP_Dataset'
activitylist= os.listdir(basedir)
activitylist= [x for x in activitylist if 
               (os.path.isdir(os.path.join(basedir, x)) and not x.startswith('.'))]
activitylist

['Descend_stairs',
 'Comb_hair',
 'Drink_glass',
 'Eat_soup',
 'Walk',
 'Sitdown_chair',
 'Climb_stairs',
 'Getup_bed',
 'Brush_teeth',
 'Liedown_bed',
 'Pour_water',
 'Use_telephone',
 'Eat_meat',
 'Standup_chair']

In [0]:
spark= SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
df= None
schema= StructType([
                    StructField('x', IntegerType(), True),
                    StructField('y', IntegerType(), True),
                    StructField('z', IntegerType(), True),
])
for category in activitylist:
  datafiles= os.listdir(os.path.join(basedir, category))
  print('Collecting', category)
  for datafile in tqdm(datafiles):
    temp_df= spark.read.option('header', 'false').option(
        'delimiter', ' ').csv(os.path.join(basedir, category, datafile), schema= schema)
    temp_df= temp_df.withColumn('class', lit(category))
    if df == None:
      df= temp_df
    else:
      df= df.union(temp_df)

In [0]:
df.show()

+---+---+---+--------------+
|  x|  y|  z|         class|
+---+---+---+--------------+
| 36| 44| 43|Descend_stairs|
| 36| 44| 43|Descend_stairs|
| 36| 38| 41|Descend_stairs|
| 36| 38| 41|Descend_stairs|
| 36| 40| 42|Descend_stairs|
| 33| 44| 45|Descend_stairs|
| 32| 41| 43|Descend_stairs|
| 32| 44| 45|Descend_stairs|
| 29| 49| 48|Descend_stairs|
| 27| 44| 44|Descend_stairs|
| 25| 53| 45|Descend_stairs|
| 22| 53| 45|Descend_stairs|
| 22| 46| 40|Descend_stairs|
| 20| 47| 39|Descend_stairs|
| 18| 48| 39|Descend_stairs|
| 19| 45| 38|Descend_stairs|
| 16| 43| 36|Descend_stairs|
| 18| 46| 37|Descend_stairs|
| 16| 44| 37|Descend_stairs|
| 16| 43| 36|Descend_stairs|
+---+---+---+--------------+
only showing top 20 rows



In [0]:
from pyspark.ml.feature import StringIndexer

indexer= StringIndexer(inputCol= 'class', outputCol= 'class_idx')
indexed= indexer.fit(df).transform(df)
indexed.show()

+---+---+---+--------------+---------+
|  x|  y|  z|         class|class_idx|
+---+---+---+--------------+---------+
| 36| 44| 43|Descend_stairs|     10.0|
| 36| 44| 43|Descend_stairs|     10.0|
| 36| 38| 41|Descend_stairs|     10.0|
| 36| 38| 41|Descend_stairs|     10.0|
| 36| 40| 42|Descend_stairs|     10.0|
| 33| 44| 45|Descend_stairs|     10.0|
| 32| 41| 43|Descend_stairs|     10.0|
| 32| 44| 45|Descend_stairs|     10.0|
| 29| 49| 48|Descend_stairs|     10.0|
| 27| 44| 44|Descend_stairs|     10.0|
| 25| 53| 45|Descend_stairs|     10.0|
| 22| 53| 45|Descend_stairs|     10.0|
| 22| 46| 40|Descend_stairs|     10.0|
| 20| 47| 39|Descend_stairs|     10.0|
| 18| 48| 39|Descend_stairs|     10.0|
| 19| 45| 38|Descend_stairs|     10.0|
| 16| 43| 36|Descend_stairs|     10.0|
| 18| 46| 37|Descend_stairs|     10.0|
| 16| 44| 37|Descend_stairs|     10.0|
| 16| 43| 36|Descend_stairs|     10.0|
+---+---+---+--------------+---------+
only showing top 20 rows



In [0]:
from pyspark.ml.feature import OneHotEncoder

encode= OneHotEncoder(inputCol= 'class_idx', outputCol= 'class_onehot')
encoded= encode.transform(indexed)
encoded.show()

+---+---+---+--------------+---------+---------------+
|  x|  y|  z|         class|class_idx|   class_onehot|
+---+---+---+--------------+---------+---------------+
| 36| 44| 43|Descend_stairs|     10.0|(13,[10],[1.0])|
| 36| 44| 43|Descend_stairs|     10.0|(13,[10],[1.0])|
| 36| 38| 41|Descend_stairs|     10.0|(13,[10],[1.0])|
| 36| 38| 41|Descend_stairs|     10.0|(13,[10],[1.0])|
| 36| 40| 42|Descend_stairs|     10.0|(13,[10],[1.0])|
| 33| 44| 45|Descend_stairs|     10.0|(13,[10],[1.0])|
| 32| 41| 43|Descend_stairs|     10.0|(13,[10],[1.0])|
| 32| 44| 45|Descend_stairs|     10.0|(13,[10],[1.0])|
| 29| 49| 48|Descend_stairs|     10.0|(13,[10],[1.0])|
| 27| 44| 44|Descend_stairs|     10.0|(13,[10],[1.0])|
| 25| 53| 45|Descend_stairs|     10.0|(13,[10],[1.0])|
| 22| 53| 45|Descend_stairs|     10.0|(13,[10],[1.0])|
| 22| 46| 40|Descend_stairs|     10.0|(13,[10],[1.0])|
| 20| 47| 39|Descend_stairs|     10.0|(13,[10],[1.0])|
| 18| 48| 39|Descend_stairs|     10.0|(13,[10],[1.0])|
| 19| 45| 

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, Normalizer

assembler= VectorAssembler(inputCols= ['x', 'y', 'z'], outputCol= 'features')
featurized= assembler.transform(encoded)
featurized.show()

+---+---+---+--------------+---------+---------------+----------------+
|  x|  y|  z|         class|class_idx|   class_onehot|        features|
+---+---+---+--------------+---------+---------------+----------------+
| 36| 44| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,44.0,43.0]|
| 36| 44| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,44.0,43.0]|
| 36| 38| 41|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,38.0,41.0]|
| 36| 38| 41|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,38.0,41.0]|
| 36| 40| 42|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,40.0,42.0]|
| 33| 44| 45|Descend_stairs|     10.0|(13,[10],[1.0])|[33.0,44.0,45.0]|
| 32| 41| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[32.0,41.0,43.0]|
| 32| 44| 45|Descend_stairs|     10.0|(13,[10],[1.0])|[32.0,44.0,45.0]|
| 29| 49| 48|Descend_stairs|     10.0|(13,[10],[1.0])|[29.0,49.0,48.0]|
| 27| 44| 44|Descend_stairs|     10.0|(13,[10],[1.0])|[27.0,44.0,44.0]|
| 25| 53| 45|Descend_stairs|     10.0|(13,[10],[1.0])|[25.0,53.0

In [0]:
normalizer= Normalizer(inputCol= 'features', outputCol= 'norm_feats')
normfeats= normalizer.transform(featurized)

normfeats.show()

+---+---+---+--------------+---------+---------------+----------------+--------------------+
|  x|  y|  z|         class|class_idx|   class_onehot|        features|          norm_feats|
+---+---+---+--------------+---------+---------------+----------------+--------------------+
| 36| 44| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,44.0,43.0]|[0.50504247348087...|
| 36| 44| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,44.0,43.0]|[0.50504247348087...|
| 36| 38| 41|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,38.0,41.0]|[0.54142990957308...|
| 36| 38| 41|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,38.0,41.0]|[0.54142990957308...|
| 36| 40| 42|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,40.0,42.0]|[0.52736286177814...|
| 33| 44| 45|Descend_stairs|     10.0|(13,[10],[1.0])|[33.0,44.0,45.0]|[0.46437437952199...|
| 32| 41| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[32.0,41.0,43.0]|[0.47419117277267...|
| 32| 44| 45|Descend_stairs|     10.0|(13,[10],[1.0])|[32.0,44.0,45.0]

In [0]:
# All of the above in a pipeline

In [0]:
from pyspark.ml import Pipeline

indexer= StringIndexer(inputCol= 'class', outputCol= 'class_idx')
encode= OneHotEncoder(inputCol= 'class_idx', outputCol= 'class_onehot')
assembler= VectorAssembler(inputCols= ['x', 'y', 'z'], outputCol= 'features')
normalizer= Normalizer(inputCol= 'features', outputCol= 'norm_feats')

pipe= Pipeline(stages= [indexer, encode, assembler, normalizer])
model= pipe.fit(df)
prediction= model.transform(df)
prediction.show()

+---+---+---+--------------+---------+---------------+----------------+--------------------+
|  x|  y|  z|         class|class_idx|   class_onehot|        features|          norm_feats|
+---+---+---+--------------+---------+---------------+----------------+--------------------+
| 36| 44| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,44.0,43.0]|[0.50504247348087...|
| 36| 44| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,44.0,43.0]|[0.50504247348087...|
| 36| 38| 41|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,38.0,41.0]|[0.54142990957308...|
| 36| 38| 41|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,38.0,41.0]|[0.54142990957308...|
| 36| 40| 42|Descend_stairs|     10.0|(13,[10],[1.0])|[36.0,40.0,42.0]|[0.52736286177814...|
| 33| 44| 45|Descend_stairs|     10.0|(13,[10],[1.0])|[33.0,44.0,45.0]|[0.46437437952199...|
| 32| 41| 43|Descend_stairs|     10.0|(13,[10],[1.0])|[32.0,41.0,43.0]|[0.47419117277267...|
| 32| 44| 45|Descend_stairs|     10.0|(13,[10],[1.0])|[32.0,44.0,45.0]

In [0]:
df_train= prediction.drop('x').drop('y').drop('z').drop('class').drop('class_idx').drop('features')
df_train.show()

+---------------+--------------------+
|   class_onehot|          norm_feats|
+---------------+--------------------+
|(13,[10],[1.0])|[0.50504247348087...|
|(13,[10],[1.0])|[0.50504247348087...|
|(13,[10],[1.0])|[0.54142990957308...|
|(13,[10],[1.0])|[0.54142990957308...|
|(13,[10],[1.0])|[0.52736286177814...|
|(13,[10],[1.0])|[0.46437437952199...|
|(13,[10],[1.0])|[0.47419117277267...|
|(13,[10],[1.0])|[0.45322869364840...|
|(13,[10],[1.0])|[0.38941086555745...|
|(13,[10],[1.0])|[0.39805001766312...|
|(13,[10],[1.0])|[0.33836346330950...|
|(13,[10],[1.0])|[0.30168138576690...|
|(13,[10],[1.0])|[0.33946736991660...|
|(13,[10],[1.0])|[0.31121102204473...|
|(13,[10],[1.0])|[0.27944785969957...|
|(13,[10],[1.0])|[0.30701119633268...|
|(13,[10],[1.0])|[0.27435739251887...|
|(13,[10],[1.0])|[0.29165338328776...|
|(13,[10],[1.0])|[0.26812295418744...|
|(13,[10],[1.0])|[0.27435739251887...|
+---------------+--------------------+
only showing top 20 rows



In [0]:
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)
 |-- class: string (nullable = false)



In [0]:
df= df.repartition(4)
df.write.parquet('df_4part.parquet')

In [0]:
df_from_objectstore= spark.read.parquet('df_4part.parquet')

In [46]:
df_from_objectstore.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)
 |-- class: string (nullable = true)



In [48]:
df_from_objectstore.show()

+---+---+---+--------------+
|  x|  y|  z|         class|
+---+---+---+--------------+
|  5| 40| 33|Descend_stairs|
|  0| 41| 36|Descend_stairs|
|  0| 40| 32|Descend_stairs|
| 18| 36| 32|Descend_stairs|
|  1| 39| 30|Descend_stairs|
|  0| 41| 32|Descend_stairs|
| 13| 39| 32|Descend_stairs|
| 36| 40| 42|Descend_stairs|
| 18| 40| 38|Descend_stairs|
| 42| 36| 51|Descend_stairs|
| 24| 38| 36|Descend_stairs|
| 11| 38| 33|Descend_stairs|
| 41| 34| 51|Descend_stairs|
| 11| 37| 31|Descend_stairs|
|  5| 40| 36|Descend_stairs|
|  1| 43| 34|Descend_stairs|
| 15| 46| 38|Descend_stairs|
|  1| 41| 31|Descend_stairs|
|  9| 41| 40|Descend_stairs|
| 23| 34| 30|Descend_stairs|
+---+---+---+--------------+
only showing top 20 rows



In [49]:
df= df_from_objectstore
df.show()

+---+---+---+--------------+
|  x|  y|  z|         class|
+---+---+---+--------------+
|  5| 40| 33|Descend_stairs|
|  0| 41| 36|Descend_stairs|
|  0| 40| 32|Descend_stairs|
| 18| 36| 32|Descend_stairs|
|  1| 39| 30|Descend_stairs|
|  0| 41| 32|Descend_stairs|
| 13| 39| 32|Descend_stairs|
| 36| 40| 42|Descend_stairs|
| 18| 40| 38|Descend_stairs|
| 42| 36| 51|Descend_stairs|
| 24| 38| 36|Descend_stairs|
| 11| 38| 33|Descend_stairs|
| 41| 34| 51|Descend_stairs|
| 11| 37| 31|Descend_stairs|
|  5| 40| 36|Descend_stairs|
|  1| 43| 34|Descend_stairs|
| 15| 46| 38|Descend_stairs|
|  1| 41| 31|Descend_stairs|
|  9| 41| 40|Descend_stairs|
| 23| 34| 30|Descend_stairs|
+---+---+---+--------------+
only showing top 20 rows



In [66]:
df.createOrReplaceTempView('df')
# spark.sql("""select sqrt(x) from df""").show()
df2= spark.sql("""select sqrt(sum(x*x) + sum(y*y) + sum(z*z)) as label,class from  df group by class""")
df2.createOrReplaceTempView('df2')

df2.show()

+------------------+--------------+
|             label|         class|
+------------------+--------------+
| 8959.680239829991| Use_telephone|
| 9737.511232342687| Standup_chair|
| 12542.96539897962|      Eat_meat|
|13225.945637269193|     Getup_bed|
|15003.269043778426|   Drink_glass|
|14454.885091207056|    Pour_water|
|10616.408809008817|     Comb_hair|
|16537.370891408344|          Walk|
|11082.626493751379|  Climb_stairs|
|10261.338314274606| Sitdown_chair|
|6783.4063714331605|   Liedown_bed|
| 7173.493500380411|Descend_stairs|
| 11785.39634462923|   Brush_teeth|
| 6071.460120926432|      Eat_soup|
+------------------+--------------+



In [67]:
df_energy= spark.sql("""select * from df inner join df2 on df.class=df2.class""")
df_energy.show()

+---+---+---+--------------+-----------------+--------------+
|  x|  y|  z|         class|            label|         class|
+---+---+---+--------------+-----------------+--------------+
|  5| 40| 33|Descend_stairs|7173.493500380411|Descend_stairs|
|  0| 41| 36|Descend_stairs|7173.493500380411|Descend_stairs|
|  0| 40| 32|Descend_stairs|7173.493500380411|Descend_stairs|
| 18| 36| 32|Descend_stairs|7173.493500380411|Descend_stairs|
|  1| 39| 30|Descend_stairs|7173.493500380411|Descend_stairs|
|  0| 41| 32|Descend_stairs|7173.493500380411|Descend_stairs|
| 13| 39| 32|Descend_stairs|7173.493500380411|Descend_stairs|
| 36| 40| 42|Descend_stairs|7173.493500380411|Descend_stairs|
| 18| 40| 38|Descend_stairs|7173.493500380411|Descend_stairs|
| 42| 36| 51|Descend_stairs|7173.493500380411|Descend_stairs|
| 24| 38| 36|Descend_stairs|7173.493500380411|Descend_stairs|
| 11| 38| 33|Descend_stairs|7173.493500380411|Descend_stairs|
| 41| 34| 51|Descend_stairs|7173.493500380411|Descend_stairs|
| 11| 37

In [71]:
## normalization
assembler= VectorAssembler(inputCols= ['x', 'y', 'z'], outputCol= 'features')
normalizer= Normalizer(inputCol= 'features', outputCol= 'norm_feats')
from pyspark.ml.regression import LinearRegression

lr= LinearRegression(maxIter= 10, regParam= 0.3, elasticNetParam= 0.8)

pipe= Pipeline(stages= [assembler, normalizer, lr])
model= pipe.fit(df_energy)
prediction= model.transform(df_energy)
prediction.show()


+---+---+---+--------------+-----------------+--------------+----------------+--------------------+------------------+
|  x|  y|  z|         class|            label|         class|        features|          norm_feats|        prediction|
+---+---+---+--------------+-----------------+--------------+----------------+--------------------+------------------+
|  5| 40| 33|Descend_stairs|7173.493500380411|Descend_stairs| [5.0,40.0,33.0]|[0.09597653852344...| 13493.10627545079|
|  0| 41| 36|Descend_stairs|7173.493500380411|Descend_stairs| [0.0,41.0,36.0]|[0.0,0.7514402250...|13764.661592173936|
|  0| 40| 32|Descend_stairs|7173.493500380411|Descend_stairs| [0.0,40.0,32.0]|[0.0,0.7808688094...|13714.128102879728|
| 18| 36| 32|Descend_stairs|7173.493500380411|Descend_stairs|[18.0,36.0,32.0]|[0.35005942871298...|12918.463327951908|
|  1| 39| 30|Descend_stairs|7173.493500380411|Descend_stairs| [1.0,39.0,30.0]|[0.02031949595270...|13648.734277921325|
|  0| 41| 32|Descend_stairs|7173.493500380411|De

In [72]:
model.stages[2].summary.r2

0.03259100556272421